# Redes Neurais Recorrentes (RNN) e LSTMs — PyTorch Puro

Este notebook ilustra o funcionamento das redes recorrentes:
- Como o estado oculto é atualizado;
- Como o PyTorch implementa `nn.RNN` e `nn.LSTM`;
- E como elas aprendem dependências temporais em sequências.

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

## 1. Recorrência passo a passo

O estado oculto `h_t` é atualizado a cada passo de tempo:

$$
h_t = f(W_{xh} x_t + W_{hh} h_{t-1} + b_h)
$$

onde `f` é uma função de ativação não linear (ex: `tanh`).

In [12]:
torch.manual_seed(0)

# Vetores de entrada (3 passos, cada um de dimensão 2)
x_seq = [torch.randn(2) for _ in range(3)]

# Parâmetros manuais
W_xh = torch.randn(2, 3)
W_hh = torch.randn(3, 3)
b_h = torch.zeros(3)

h = torch.zeros(3)
print("Estado inicial:", h)

for i, x_t in enumerate(x_seq):
    h = torch.tanh(x_t @ W_xh + h @ W_hh + b_h)
    print(f"h_{i+1} =", h)

Estado inicial: tensor([0., 0., 0.])
h_1 = tensor([ 0.6291,  0.8989, -0.8216])
h_2 = tensor([-0.6662, -0.6302,  0.7730])
h_3 = tensor([-0.0694, -0.9242,  0.8592])


## 2. Usando `nn.RNN` no PyTorch

O módulo `nn.RNN` faz automaticamente o mesmo processo.

In [13]:
rnn = nn.RNN(input_size=2, hidden_size=3, batch_first=True)

# Uma sequência com batch_size=1 e seq_len=3
x = torch.stack(x_seq).unsqueeze(0)
out, h_final = rnn(x)

print("Saída (por passo):", out)
print("Estado final:", h_final)

Saída (por passo): tensor([[[-0.8324,  0.7152,  0.1593],
         [ 0.2347, -0.0078,  0.5524],
         [ 0.4113, -0.5408,  0.5808]]], grad_fn=<TransposeBackward1>)
Estado final: tensor([[[ 0.4113, -0.5408,  0.5808]]], grad_fn=<StackBackward0>)


## 3. Treinando uma RNN simples

Vamos criar uma tarefa de previsão de próxima letra em uma sequência curta.
O objetivo é prever a próxima letra da palavra "ola".

In [14]:
# mapeia caracteres
char_to_idx = {"o": 0, "l": 1, "a": 2}
idx_to_char = {i: c for c, i in char_to_idx.items()}

seq = [0, 1, 2]  # o, l, a
x_data = torch.tensor([[0, 1]], dtype=torch.long)  # "ol"
y_data = torch.tensor([2], dtype=torch.long)       # "a"

# Modelo simples
class CharRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        emb = self.emb(x)
        out, _ = self.rnn(emb)
        out = self.fc(out[:, -1, :])  # usa último passo
        return out

model = CharRNN(vocab_size=3, hidden_size=8)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

for epoch in range(50):
    optimizer.zero_grad()
    output = model(x_data)
    loss = criterion(output, y_data)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 5 == 0:
        print(f"Época {epoch+1} | Loss: {loss.item():.4f}")

pred = model(x_data).argmax(dim=1).item()
print(f"Entrada: 'ol' → Previsão: '{idx_to_char[pred]}'")

Época 5 | Loss: 0.1912
Época 10 | Loss: 0.0106
Época 15 | Loss: 0.0017
Época 20 | Loss: 0.0006
Época 25 | Loss: 0.0003
Época 30 | Loss: 0.0002
Época 35 | Loss: 0.0001
Época 40 | Loss: 0.0001
Época 45 | Loss: 0.0001
Época 50 | Loss: 0.0001
Entrada: 'ol' → Previsão: 'a'


## 4. Usando `nn.LSTM`

A LSTM (Long Short-Term Memory) introduz *portas* que controlam o fluxo de informação.
Ela é mais estável que a RNN em sequências longas.

In [15]:
lstm = nn.LSTM(input_size=2, hidden_size=3, batch_first=True)
x = torch.stack(x_seq).unsqueeze(0)
out, (h_n, c_n) = lstm(x)

print("Saída (LSTM):", out)
print("Estado oculto final:", h_n)
print("Estado de célula (memória interna):", c_n)

Saída (LSTM): tensor([[[ 0.1721, -0.0125, -0.1078],
         [ 0.2675,  0.1160, -0.2038],
         [ 0.4472,  0.2182, -0.3425]]], grad_fn=<TransposeBackward0>)
Estado oculto final: tensor([[[ 0.4472,  0.2182, -0.3425]]], grad_fn=<StackBackward0>)
Estado de célula (memória interna): tensor([[[ 0.6073,  0.5507, -0.6291]]], grad_fn=<StackBackward0>)


## 5. Comparação RNN vs LSTM

- **RNN:** propaga apenas o estado oculto `h_t`; tende a sofrer com *vanishing gradients*.
- **LSTM:** mantém também um estado de célula `c_t`, que preserva melhor informações de longo prazo.

LSTMs são preferidas na maioria das tarefas de sequência mais longas (tradução, fala, etc.).

---

> **Resumo:** RNNs capturam dependências locais em sequências; LSTMs ampliam essa capacidade com memória controlada por portas.